<a href="https://colab.research.google.com/github/Chubbyman2/Resume/blob/master/VGG16_Paper_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"Very Deep Convolutional Networks for Large-Scale Image Recognition" 

by K. Simonyan and A. Zisserman

Link: https://arxiv.org/pdf/1409.1556.pdf

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [4]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
# CIFAR dataset has 50,000 training samples and 10,000 test samples
validation_images, validation_labels = train_images[:5000], train_labels[:5000]
train_images, train_labels = train_images[5000:], train_labels[5000:]

In [6]:
# Takes the train, test, val datasets
# Returns corresponding tf Dataset representations
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [7]:
# Paper detailed batch size of 256, but Colab did not have enough RAM
BATCH_SIZE = 128
IMG_SHAPE = 224

def process_images(image, label):
    # Normalize images
    image = tf.image.per_image_standardization(image)

    # Resize images
    image = tf.image.resize(image, (IMG_SHAPE, IMG_SHAPE))

    return image, label

In [8]:
# Get size of each dataset
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

In [9]:
# Preprocess data, shuffle dataset, put data into batches
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=BATCH_SIZE, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=BATCH_SIZE, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=BATCH_SIZE, drop_remainder=True))

In [10]:
# Build model
# No batch normalization, as described in paper
VGG16 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64, input_shape=(IMG_SHAPE, IMG_SHAPE, 3), kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [11]:
# Compile model
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
VGG16.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
VGG16.summary()

In [13]:
# Configure checkpoints
checkpoint_dir = "./training_checkpoints"

# Name of checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [ ]:
# Train model
EPOCHS = 74

# Decrease the learning rate by factor 10 when val_accuracy stops improving
decrease_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.1, patience=1, verbose=0, mode='auto'
)

history = VGG16.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=validation_ds,
    callbacks=[decrease_lr, checkpoint_callback]
)

In [ ]:
# Final test
VGG16.evaluate(test_ds)